In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("data.csv")
raw_df.drop_duplicates()
display(raw_df)

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,will_go_to_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,True,True
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,False,False
...,...,...,...,...,...,...,...,...,...,...,...
995,Vocational,A,Female,Very Interested,Rural,49,7420000,63.6,85.99,True,True
996,Academic,B,Female,Less Interested,Rural,51,7480000,84.3,89.72,True,True
997,Vocational,A,Male,Less Interested,Urban,49,5550000,75.2,79.56,False,True
998,Academic,B,Male,Uncertain,Rural,53,5840000,105.8,87.18,True,True


In [3]:
man_df=raw_df.copy().loc[raw_df["gender"]=="Male"].loc[:,['gender','will_go_to_college']]
woman_df=raw_df.copy().loc[raw_df["gender"]!="Male"].loc[:,['gender','will_go_to_college']]

def get_percentage(value):
    return round(value,4)*100

print(f"The privillege gorups take up {get_percentage(man_df.shape[0]/raw_df.shape[0])}% samples")
print(f"The unprivillege gorups take up {get_percentage(woman_df.shape[0]/raw_df.shape[0])}% samples")

man_label=man_df["will_go_to_college"].value_counts()
print(f"The privillege groups have {get_percentage(man_label[1]/man_df.shape[0])}% positive label")
print(f"The privillege groups have {get_percentage(man_label[0]/man_df.shape[0])}% negative lablel")

women_label=woman_df["will_go_to_college"].value_counts()
print(f"The unprivillege groups have {get_percentage(women_label[1]/woman_df.shape[0])}% positive label")
print(f"The unprivillege groups have {get_percentage(women_label[0]/woman_df.shape[0])}% negative lablel")

The privillege gorups take up 51.5% samples
The unprivillege gorups take up 48.5% samples
The privillege groups have 48.35% positive label
The privillege groups have 51.65% negative lablel
The unprivillege groups have 48.25% positive label
The unprivillege groups have 51.74999999999999% negative lablel


In [4]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

df=raw_df.copy(deep=True)
#normalize PreviousSalary feature
df["parent_age"]=min_max_normalisation(df["parent_age"])
df["parent_salary"]=min_max_normalisation(df["parent_salary"])
df["house_area"]=min_max_normalisation(df["house_area"])
df["average_grades"]=min_max_normalisation(df["average_grades"])

display(df)

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,will_go_to_college
0,Academic,A,Male,Less Interested,Urban,0.64,0.661111,0.630,0.395217,False,True
1,Academic,A,Male,Less Interested,Urban,0.68,0.378889,0.568,0.517826,False,True
2,Academic,B,Female,Very Interested,Urban,0.40,0.611111,0.606,0.540435,False,True
3,Vocational,B,Male,Very Interested,Rural,0.36,0.622222,0.582,0.309565,True,True
4,Academic,A,Female,Very Interested,Urban,0.68,0.472222,0.551,0.512609,False,False
...,...,...,...,...,...,...,...,...,...,...,...
995,Vocational,A,Female,Very Interested,Rural,0.36,0.713333,0.436,0.477826,True,True
996,Academic,B,Female,Less Interested,Rural,0.44,0.720000,0.643,0.640000,True,True
997,Vocational,A,Male,Less Interested,Urban,0.36,0.505556,0.552,0.198261,False,True
998,Academic,B,Male,Uncertain,Rural,0.52,0.537778,0.858,0.529565,True,True


In [5]:
folds_index=np.array_split(df.sample(frac=1).index,5)
raw_dataset=StandardDataset(df=df,label_name="will_go_to_college",favorable_classes=[True],scores_name=""
                        ,protected_attribute_names=["gender"],privileged_classes=[["Male"]]
                        ,categorical_features=['type_school',"school_accreditation","interest","residence","parent_was_in_college"])

In [6]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
Precision_data=pd.DataFrame(columns=Cross_Validation_Column)
Recall_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"gender":1}]
unprivileged_groups = [{"gender":0}]

In [7]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global Precision_data
    global Recall_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    precision_row=[]
    recall_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for index in range(5):
        test_fold=folds_index[index]
        train_index=pd.Index([])
        for f in folds_index:
            if f is not test_fold:
                train_index=train_index.append(f)
        train_set=raw_dataset.subset(train_index)
        test_set=raw_dataset.subset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        precision_row.append(metric.precision())
        recall_row.append(metric.recall())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    precision_row.append(np.average(precision_row))
    recall_row.append(np.average(recall_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    Precision_data=pd.concat([Precision_data,pd.DataFrame([precision_row],index=[row_name],columns=Cross_Validation_Column)])
    Precision_data.columns.name="Precision"
    display(Precision_data)
    Recall_data=pd.concat([Recall_data,pd.DataFrame([recall_row],index=[row_name],columns=Cross_Validation_Column)])
    Recall_data.columns.name="Recall"
    display(Recall_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)


In [8]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.89,0.84,0.91,0.898


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.92,0.891089,0.825243,0.914894,0.896784


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.16388,1.062422,1.104167,1.045355,1.067327,1.08863


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.05,0.054902,0.046816


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.01617,-0.025926,0.0,0.024823,-0.071111,-0.017677


In [9]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.695073; batch adversarial loss: 0.696296
epoch 1; iter: 0; batch classifier loss: 0.659788; batch adversarial loss: 0.690476
epoch 2; iter: 0; batch classifier loss: 0.670571; batch adversarial loss: 0.705608
epoch 3; iter: 0; batch classifier loss: 0.636965; batch adversarial loss: 0.703365
epoch 4; iter: 0; batch classifier loss: 0.638916; batch adversarial loss: 0.694274
epoch 5; iter: 0; batch classifier loss: 0.628626; batch adversarial loss: 0.695840
epoch 6; iter: 0; batch classifier loss: 0.595673; batch adversarial loss: 0.701791
epoch 7; iter: 0; batch classifier loss: 0.584778; batch adversarial loss: 0.687325
epoch 8; iter: 0; batch classifier loss: 0.575579; batch adversarial loss: 0.691974
epoch 9; iter: 0; batch classifier loss: 0.579591; batch adversarial loss: 0.708609
epoch 10; iter: 0; batch classifier loss: 0.528446; batch adversarial loss: 0.701513
epoch 11; iter: 0; batch classifier loss: 0.581748; batch adversarial loss:

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.91,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.88,0.863


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771


In [10]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307


In [11]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225


In [12]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603


In [13]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751


In [14]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905
DisparateImpactRemover(0.7),0.895,0.920,0.895,0.885,0.925,0.904


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189
DisparateImpactRemover(0.7),0.927083,0.947368,0.908163,0.872549,0.935484,0.918130


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845
DisparateImpactRemover(0.7),0.864078,0.891089,0.881188,0.898990,0.906250,0.888319


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562
DisparateImpactRemover(0.7),1.137746,1.122472,1.085106,1.023576,1.139765,1.101733


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351
DisparateImpactRemover(0.7),0.061699,0.055168,0.04,0.011896,0.060706,0.045894


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132
DisparateImpactRemover(0.7),0.029367,0.152711,-0.018577,-0.022727,0.074510,0.043057


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751
DisparateImpactRemover(0.7),-0.016170,-0.011111,-0.044444,0.077620,-0.034074,-0.005636


In [15]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905
DisparateImpactRemover(0.7),0.895,0.920,0.895,0.885,0.925,0.904
DisparateImpactRemover(1.0),0.925,0.930,0.900,0.890,0.920,0.913


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189
DisparateImpactRemover(0.7),0.927083,0.947368,0.908163,0.872549,0.935484,0.918130
DisparateImpactRemover(1.0),0.940000,0.957895,0.926316,0.853211,0.954545,0.926393


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845
DisparateImpactRemover(0.7),0.864078,0.891089,0.881188,0.898990,0.906250,0.888319
DisparateImpactRemover(1.0),0.912621,0.900990,0.871287,0.939394,0.875000,0.899859


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562
DisparateImpactRemover(0.7),1.137746,1.122472,1.085106,1.023576,1.139765,1.101733
DisparateImpactRemover(1.0),1.128205,1.076008,1.261905,0.942533,1.176238,1.116978


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351
DisparateImpactRemover(0.7),0.061699,0.055168,0.04,0.011896,0.060706,0.045894
DisparateImpactRemover(1.0),0.060096,0.034923,0.11,-0.032161,0.071207,0.048813


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132
DisparateImpactRemover(0.7),0.029367,0.152711,-0.018577,-0.022727,0.074510,0.043057
DisparateImpactRemover(1.0),-0.000763,0.135167,0.083004,-0.054545,0.099346,0.052442


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751
DisparateImpactRemover(0.7),-0.016170,-0.011111,-0.044444,0.077620,-0.034074,-0.005636
DisparateImpactRemover(1.0),0.003830,-0.033333,-0.007407,0.022065,-0.035556,-0.010080


In [16]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905
DisparateImpactRemover(0.7),0.895,0.920,0.895,0.885,0.925,0.904
DisparateImpactRemover(1.0),0.925,0.930,0.900,0.890,0.920,0.913
CalibratedEqOddsPostprocessing,0.935,0.920,0.895,0.860,0.915,0.905


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189
DisparateImpactRemover(0.7),0.927083,0.947368,0.908163,0.872549,0.935484,0.918130
DisparateImpactRemover(1.0),0.940000,0.957895,0.926316,0.853211,0.954545,0.926393
CalibratedEqOddsPostprocessing,0.932692,0.938144,0.877358,0.838095,0.943820,0.906022


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845
DisparateImpactRemover(0.7),0.864078,0.891089,0.881188,0.898990,0.906250,0.888319
DisparateImpactRemover(1.0),0.912621,0.900990,0.871287,0.939394,0.875000,0.899859
CalibratedEqOddsPostprocessing,0.941748,0.900990,0.920792,0.888889,0.875000,0.905484


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562
DisparateImpactRemover(0.7),1.137746,1.122472,1.085106,1.023576,1.139765,1.101733
DisparateImpactRemover(1.0),1.128205,1.076008,1.261905,0.942533,1.176238,1.116978
CalibratedEqOddsPostprocessing,1.163880,1.035404,1.078431,1.048077,1.096888,1.084536


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351
DisparateImpactRemover(0.7),0.061699,0.055168,0.04,0.011896,0.060706,0.045894
DisparateImpactRemover(1.0),0.060096,0.034923,0.11,-0.032161,0.071207,0.048813
CalibratedEqOddsPostprocessing,0.078526,0.016905,0.04,0.024700,0.041104,0.040247


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132
DisparateImpactRemover(0.7),0.029367,0.152711,-0.018577,-0.022727,0.074510,0.043057
DisparateImpactRemover(1.0),-0.000763,0.135167,0.083004,-0.054545,0.099346,0.052442
CalibratedEqOddsPostprocessing,0.051869,0.094896,-0.025692,0.036364,0.015686,0.034625


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751
DisparateImpactRemover(0.7),-0.016170,-0.011111,-0.044444,0.077620,-0.034074,-0.005636
DisparateImpactRemover(1.0),0.003830,-0.033333,-0.007407,0.022065,-0.035556,-0.010080
CalibratedEqOddsPostprocessing,-0.016170,-0.029630,-0.037037,0.043341,-0.015556,-0.011010


In [17]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905
DisparateImpactRemover(0.7),0.895,0.920,0.895,0.885,0.925,0.904
DisparateImpactRemover(1.0),0.925,0.930,0.900,0.890,0.920,0.913
CalibratedEqOddsPostprocessing,0.935,0.920,0.895,0.860,0.915,0.905
RejectOptionClassification(EO),0.935,0.930,0.900,0.845,0.930,0.908


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189
DisparateImpactRemover(0.7),0.927083,0.947368,0.908163,0.872549,0.935484,0.918130
DisparateImpactRemover(1.0),0.940000,0.957895,0.926316,0.853211,0.954545,0.926393
CalibratedEqOddsPostprocessing,0.932692,0.938144,0.877358,0.838095,0.943820,0.906022
RejectOptionClassification(EO),0.941176,0.939394,0.885714,0.820755,0.955556,0.908519


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845
DisparateImpactRemover(0.7),0.864078,0.891089,0.881188,0.898990,0.906250,0.888319
DisparateImpactRemover(1.0),0.912621,0.900990,0.871287,0.939394,0.875000,0.899859
CalibratedEqOddsPostprocessing,0.941748,0.900990,0.920792,0.888889,0.875000,0.905484
RejectOptionClassification(EO),0.932039,0.920792,0.920792,0.878788,0.895833,0.909649


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562
DisparateImpactRemover(0.7),1.137746,1.122472,1.085106,1.023576,1.139765,1.101733
DisparateImpactRemover(1.0),1.128205,1.076008,1.261905,0.942533,1.176238,1.116978
CalibratedEqOddsPostprocessing,1.163880,1.035404,1.078431,1.048077,1.096888,1.084536
RejectOptionClassification(EO),1.216783,1.039326,1.142857,0.991285,1.071379,1.092326


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351
DisparateImpactRemover(0.7),0.061699,0.055168,0.04,0.011896,0.060706,0.045894
DisparateImpactRemover(1.0),0.060096,0.034923,0.11,-0.032161,0.071207,0.048813
CalibratedEqOddsPostprocessing,0.078526,0.016905,0.04,0.024700,0.041104,0.040247
RejectOptionClassification(EO),0.099359,0.019131,0.07,-0.004638,0.031003,0.042971


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132
DisparateImpactRemover(0.7),0.029367,0.152711,-0.018577,-0.022727,0.074510,0.043057
DisparateImpactRemover(1.0),-0.000763,0.135167,0.083004,-0.054545,0.099346,0.052442
CalibratedEqOddsPostprocessing,0.051869,0.094896,-0.025692,0.036364,0.015686,0.034625
RejectOptionClassification(EO),0.073608,0.100080,0.014229,0.013636,0.013072,0.042925


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751
DisparateImpactRemover(0.7),-0.016170,-0.011111,-0.044444,0.077620,-0.034074,-0.005636
DisparateImpactRemover(1.0),0.003830,-0.033333,-0.007407,0.022065,-0.035556,-0.010080
CalibratedEqOddsPostprocessing,-0.016170,-0.029630,-0.037037,0.043341,-0.015556,-0.011010
RejectOptionClassification(EO),0.003830,-0.029630,-0.018519,0.006304,-0.035556,-0.014714


In [18]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.935,0.915,0.890,0.840,0.910,0.898
AdversarialDebiase,0.875,0.800,0.905,0.855,0.880,0.863
GridSearchReduction(EO),0.920,0.930,0.925,0.855,0.925,0.911
GridSearchReduction(DP),0.920,0.930,0.895,0.855,0.910,0.902
Reweight,0.935,0.920,0.910,0.845,0.930,0.908
DisparateImpactRemover(0.5),0.925,0.915,0.895,0.860,0.930,0.905
DisparateImpactRemover(0.7),0.895,0.920,0.895,0.885,0.925,0.904
DisparateImpactRemover(1.0),0.925,0.930,0.900,0.890,0.920,0.913
CalibratedEqOddsPostprocessing,0.935,0.920,0.895,0.860,0.915,0.905
RejectOptionClassification(EO),0.935,0.930,0.900,0.845,0.930,0.908


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.932692,0.920000,0.891089,0.825243,0.914894,0.896784
AdversarialDebiase,0.897959,0.765217,0.918367,0.857143,0.891304,0.865998
GridSearchReduction(EO),0.930693,0.939394,0.913462,0.824074,0.935484,0.908621
GridSearchReduction(DP),0.930693,0.948454,0.884615,0.843137,0.923913,0.906162
Reweight,0.950000,0.938144,0.902913,0.826923,0.965909,0.916778
DisparateImpactRemover(0.5),0.940000,0.928571,0.892157,0.844660,0.955556,0.912189
DisparateImpactRemover(0.7),0.927083,0.947368,0.908163,0.872549,0.935484,0.918130
DisparateImpactRemover(1.0),0.940000,0.957895,0.926316,0.853211,0.954545,0.926393
CalibratedEqOddsPostprocessing,0.932692,0.938144,0.877358,0.838095,0.943820,0.906022
RejectOptionClassification(EO),0.941176,0.939394,0.885714,0.820755,0.955556,0.908519


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.941748,0.910891,0.891089,0.858586,0.895833,0.899629
AdversarialDebiase,0.854369,0.871287,0.891089,0.848485,0.854167,0.863879
GridSearchReduction(EO),0.912621,0.920792,0.940594,0.898990,0.906250,0.915849
GridSearchReduction(DP),0.912621,0.910891,0.910891,0.868687,0.885417,0.897701
Reweight,0.922330,0.900990,0.920792,0.868687,0.885417,0.899643
DisparateImpactRemover(0.5),0.912621,0.900990,0.900990,0.878788,0.895833,0.897845
DisparateImpactRemover(0.7),0.864078,0.891089,0.881188,0.898990,0.906250,0.888319
DisparateImpactRemover(1.0),0.912621,0.900990,0.871287,0.939394,0.875000,0.899859
CalibratedEqOddsPostprocessing,0.941748,0.900990,0.920792,0.888889,0.875000,0.905484
RejectOptionClassification(EO),0.932039,0.920792,0.920792,0.878788,0.895833,0.909649


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.163880,1.062422,1.104167,1.045355,1.067327,1.088630
AdversarialDebiase,1.283302,0.523512,0.960000,1.197802,1.392913,1.071506
GridSearchReduction(EO),1.103679,1.039326,1.080000,0.958242,1.045545,1.045358
GridSearchReduction(DP),1.060556,1.079300,1.166667,0.945633,1.069307,1.064293
Reweight,1.223614,1.035404,1.191489,1.026688,1.123642,1.120167
DisparateImpactRemover(0.5),1.174825,1.016230,1.125000,1.005298,1.171456,1.098562
DisparateImpactRemover(0.7),1.137746,1.122472,1.085106,1.023576,1.139765,1.101733
DisparateImpactRemover(1.0),1.128205,1.076008,1.261905,0.942533,1.176238,1.116978
CalibratedEqOddsPostprocessing,1.163880,1.035404,1.078431,1.048077,1.096888,1.084536
RejectOptionClassification(EO),1.216783,1.039326,1.142857,0.991285,1.071379,1.092326


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.078526,0.030367,0.05,0.022885,0.030603,0.042476
AdversarialDebiase,0.120994,-0.347707,-0.02,0.088920,0.150815,-0.001396
GridSearchReduction(EO),0.049679,0.019131,0.04,-0.022986,0.020702,0.021305
GridSearchReduction(DP),0.029647,0.037150,0.08,-0.028430,0.030803,0.029834
Reweight,0.100160,0.016905,0.09,0.013711,0.051205,0.054396
DisparateImpactRemover(0.5),0.080128,0.007896,0.06,0.002722,0.071007,0.044351
DisparateImpactRemover(0.7),0.061699,0.055168,0.04,0.011896,0.060706,0.045894
DisparateImpactRemover(1.0),0.060096,0.034923,0.11,-0.032161,0.071207,0.048813
CalibratedEqOddsPostprocessing,0.078526,0.016905,0.04,0.024700,0.041104,0.040247
RejectOptionClassification(EO),0.099359,0.019131,0.07,-0.004638,0.031003,0.042971


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.051869,0.117624,-0.040316,0.050000,0.054902,0.046816
AdversarialDebiase,0.051106,-0.255183,-0.080237,0.068182,0.143791,-0.014468
GridSearchReduction(EO),-0.000763,0.100080,-0.029249,-0.022727,0.032680,0.016004
GridSearchReduction(DP),-0.040046,0.117624,0.035968,-0.009091,0.035294,0.027950
Reweight,0.056064,0.094896,0.014229,0.031818,0.077124,0.054826
DisparateImpactRemover(0.5),0.038520,0.094896,0.017787,-0.027273,0.096732,0.044132
DisparateImpactRemover(0.7),0.029367,0.152711,-0.018577,-0.022727,0.074510,0.043057
DisparateImpactRemover(1.0),-0.000763,0.135167,0.083004,-0.054545,0.099346,0.052442
CalibratedEqOddsPostprocessing,0.051869,0.094896,-0.025692,0.036364,0.015686,0.034625
RejectOptionClassification(EO),0.073608,0.100080,0.014229,0.013636,0.013072,0.042925


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.016170,-0.025926,0.000000,0.024823,-0.071111,-0.017677
AdversarialDebiase,0.088936,-0.418519,-0.107407,0.138692,0.084444,-0.042771
GridSearchReduction(EO),-0.016170,-0.029630,-0.044444,0.006304,-0.072593,-0.031307
GridSearchReduction(DP),-0.016170,-0.011111,-0.018519,-0.017730,-0.052593,-0.023225
Reweight,0.023830,-0.029630,0.018519,0.024823,-0.055556,-0.003603
DisparateImpactRemover(0.5),0.003830,-0.048148,-0.040741,0.061860,-0.035556,-0.011751
DisparateImpactRemover(0.7),-0.016170,-0.011111,-0.044444,0.077620,-0.034074,-0.005636
DisparateImpactRemover(1.0),0.003830,-0.033333,-0.007407,0.022065,-0.035556,-0.010080
CalibratedEqOddsPostprocessing,-0.016170,-0.029630,-0.037037,0.043341,-0.015556,-0.011010
RejectOptionClassification(EO),0.003830,-0.029630,-0.018519,0.006304,-0.035556,-0.014714


In [19]:
Accuracy_data.to_csv("college_accuracy.csv")
Precision_data.to_csv("college_precision.csv")
Recall_data.to_csv("college_recall.csv")
DisparateImpact_data.to_csv("college_disparate_impact.csv")
DisparateParity_data.to_csv("college_disparate_parity.csv")
EqualOpportunity_data.to_csv("college_equal_opportunity.csv")
EqualizedOdds_data.to_csv("college_equalized_odds.csv")